In [2]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils
from sklearn.model_selection import StratifiedKFold

2023-03-22 17:03:24.667419: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
d = pd.concat([pd.read_csv("data/Youtube01-Psy.csv"),
               pd.read_csv("data/Youtube02-KatyPerry.csv"),
               pd.read_csv("data/Youtube03-LMFAO.csv"),
               pd.read_csv("data/Youtube04-Eminem.csv"),
               pd.read_csv("data/Youtube05-Shakira.csv")])
d = d.sample(frac=1)

In [4]:
kfold = StratifiedKFold(n_splits=5)
splits = kfold.split(d, d['CLASS'])

In [5]:
for train, test in splits:
    print("Split")
    print(test)

Split
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 

In [6]:
def train_and_test(train_idx, test_idx):
    
    train_content = d['CONTENT'].iloc[train_idx]
    test_content = d['CONTENT'].iloc[test_idx]
    
    tokenizer = Tokenizer(num_words=2000)
    
    # learn the training words (not the testing words!)
    tokenizer.fit_on_texts(train_content)

    # options for mode: binary, freq, tfidf
    d_train_inputs = tokenizer.texts_to_matrix(train_content, mode='tfidf')
    d_test_inputs = tokenizer.texts_to_matrix(test_content, mode='tfidf')

    # divide tfidf by max
    d_train_inputs = d_train_inputs/np.amax(np.absolute(d_train_inputs))
    d_test_inputs = d_test_inputs/np.amax(np.absolute(d_test_inputs))

    # subtract mean, to get values between -1 and 1
    d_train_inputs = d_train_inputs - np.mean(d_train_inputs)
    d_test_inputs = d_test_inputs - np.mean(d_test_inputs)

    # one-hot encoding of outputs
    d_train_outputs = np_utils.to_categorical(d['CLASS'].iloc[train_idx])
    d_test_outputs = np_utils.to_categorical(d['CLASS'].iloc[test_idx])

    model = Sequential()
    model.add(Dense(512, input_shape=(2000,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adamax',
                  metrics=['accuracy'])

    model.fit(d_train_inputs, d_train_outputs, epochs=10, batch_size=16)

    scores = model.evaluate(d_test_inputs, d_test_outputs)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    return scores

In [7]:
kfold = StratifiedKFold(n_splits=5)
splits = kfold.split(d, d['CLASS'])
cvscores = []
for train_idx, test_idx, in splits:
    scores = train_and_test(train_idx, test_idx)
    cvscores.append(scores[1] * 100)

Epoch 1/10


2023-03-22 17:03:37.789421: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 17:03:37.790720: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


98/98 [==============================] - 2s 13ms/step - loss: 0.6524 - accuracy: 0.7692
Epoch 2/10
98/98 [==============================] - 1s 11ms/step - loss: 0.5388 - accuracy: 0.8817
Epoch 3/10
98/98 [==============================] - 1s 12ms/step - loss: 0.4191 - accuracy: 0.9054
Epoch 4/10
98/98 [==============================] - 1s 13ms/step - loss: 0.3196 - accuracy: 0.9386
Epoch 5/10
98/98 [==============================] - 1s 15ms/step - loss: 0.2504 - accuracy: 0.9399
Epoch 6/10
98/98 [==============================] - 1s 13ms/step - loss: 0.2084 - accuracy: 0.9501
Epoch 7/10
98/98 [==============================] - 1s 12ms/step - loss: 0.1779 - accuracy: 0.9572
Epoch 8/10
98/98 [==============================] - 2s 21ms/step - loss: 0.1571 - accuracy: 0.9572
Epoch 9/10
98/98 [==============================] - 2s 17ms/step - loss: 0.1405 - accuracy: 0.9610
Epoch 10/10
13/13 [==============================] - 0s 7ms/step - loss: 0.1406 - accuracy: 0.9464
accuracy: 94.64%
Epoc

In [8]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

94.68% (+/- 0.41%)
